In [8]:
from sqlalchemy import create_engine, text
import pandas as pd
db_path = r'C:\Users\vivia\co-pilot-v1\Notebooks\bulk_upload.db'

engine = create_engine(f'sqlite:///{db_path}')

with engine.connect() as connection:
    query = text("""SELECT model_name FROM models;"""
                            )
    df = connection.execute(query)
df = pd.DataFrame(df.fetchall())
df

,model_name
0,Accord
1,CR-V
2,CR-Z
3,Civic
4,Clarity
5,Crosstour
6,Element
7,Fit
8,HR-V
9,Insight
